In [1]:
from IPython.lib.pretty import pprint
import logging
logger = logging.getLogger('gutils')
logger.handlers = [logging.StreamHandler()]
logger.setLevel(logging.DEBUG)

# Load a DataFrame of standardized data

In [2]:
from pathlib import Path
from gutils.slocum import SlocumReader

ascii_folder = Path('.').absolute().parent.parent / 'gutils' / 'tests' / 'resources' / 'slocum'
ascii_file = ascii_folder / 'usf_bass_2016_252_1_12_sbd.dat'
slocum_data = SlocumReader(str(ascii_file))
standard = slocum_data.standardize()
standard.head(20)

,c_heading,c_wpt_lat,m_altitude,m_avg_speed,m_ballast_pumped,m_battery,m_battpos,m_depth,m_depth_rate,m_gps_lat,...,temperature,t,drv_m_gps_lat,drv_m_gps_lon,y,x,pressure,z,salinity,density
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,2016-09-09 16:50:20.523320,NaN,NaN,28.366750,-80.295853,0.000000,0.000000,0.000000,999.843073
1,0.733753,28.5,16.1013,0.2054,226.820,15.5397,0.970180,0.304552,-0.002276,28.366750,...,NaN,2016-09-09 16:50:37.544100,28.366750,-80.295853,28.366750,-80.295853,0.306599,0.304552,NaN,NaN
2,NaN,NaN,NaN,NaN,226.323,NaN,0.972148,0.308441,NaN,NaN,...,NaN,2016-09-09 16:50:37.544100,NaN,NaN,28.366750,-80.295853,0.310514,0.308441,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:51:35.388820,NaN,NaN,28.367303,-80.297225,0.000000,0.000000,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:51:40.490420,NaN,NaN,28.367351,-80.297344,0.000000,0.000000,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:51:45.578460,NaN,NaN,28.367398,-80.297462,0.000000,0.000000,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:51:50.667110,NaN,NaN,28.367446,-80.297580,0.000000,0.000000,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:51:55.768830,NaN,NaN,28.367494,-80.297698,0.000000,0.000000,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:52:04.789000,NaN,NaN,28.367580,-80.297911,0.000000,0.000000,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,2016-09-09 16:52:09.880830,NaN,NaN,28.367627,-80.298030,0.000000,0.000000,NaN,NaN


In [3]:
standard.tail(20)[[
    't',
    'y',
    'x',
    'pressure'
]]

,t,y,x,pressure
702,2016-09-09 17:15:14.434390,28.370813,-80.297089,0.016838
703,2016-09-09 17:15:19.617980,28.370819,-80.297100,0.024669
704,2016-09-09 17:15:24.801610,28.370826,-80.297111,0.024669
705,2016-09-09 17:15:29.980260,28.370832,-80.297122,0.040332
706,2016-09-09 17:15:35.169160,28.370839,-80.297136,0.000000
707,2016-09-09 17:15:40.381320,28.370846,-80.297147,0.040332
708,2016-09-09 17:15:45.507200,28.370852,-80.297158,0.353587
709,2016-09-09 17:15:50.620850,28.370852,-80.297158,0.529793
710,2016-09-09 17:15:55.725130,28.370852,-80.297158,0.341840
711,2016-09-09 17:16:00.834440,28.370852,-80.297158,0.349672


## Assign profiles

In [4]:
from gutils.yo import assign_profiles

# Defaults to a time interval of 10
profiled_1 = assign_profiles(standard)
print('Profiles : ', len(profiled_1.profile.unique()))
profiled_1.tail(20)[[
    't',
    'y',
    'x',
    'pressure',
    'profile'
]]

Profiles :  18


,t,y,x,pressure,profile
702,2016-09-09 17:15:14.434390,28.370813,-80.297089,0.016838,17.0
703,2016-09-09 17:15:19.617980,28.370819,-80.297100,0.024669,17.0
704,2016-09-09 17:15:24.801610,28.370826,-80.297111,0.024669,17.0
705,2016-09-09 17:15:29.980260,28.370832,-80.297122,0.040332,17.0
706,2016-09-09 17:15:35.169160,28.370839,-80.297136,0.000000,17.0
707,2016-09-09 17:15:40.381320,28.370846,-80.297147,0.040332,17.0
708,2016-09-09 17:15:45.507200,28.370852,-80.297158,0.353587,17.0
709,2016-09-09 17:15:50.620850,28.370852,-80.297158,0.529793,18.0
710,2016-09-09 17:15:55.725130,28.370852,-80.297158,0.341840,18.0
711,2016-09-09 17:16:00.834440,28.370852,-80.297158,0.349672,19.0


In [5]:
# Shorten the time window for a profile to 2 seconds
profiled_2 = assign_profiles(standard, tsint=2)
print('Profiles : ', len(profiled_2.profile.unique()))
profiled_2.tail(20)[[
    't',
    'y',
    'x',
    'pressure',
    'profile'
]]

Profiles :  85


,t,y,x,pressure,profile
702,2016-09-09 17:15:14.434390,28.370813,-80.297089,0.016838,77.0
703,2016-09-09 17:15:19.617980,28.370819,-80.297100,0.024669,77.0
704,2016-09-09 17:15:24.801610,28.370826,-80.297111,0.024669,78.0
705,2016-09-09 17:15:29.980260,28.370832,-80.297122,0.040332,79.0
706,2016-09-09 17:15:35.169160,28.370839,-80.297136,0.000000,81.0
707,2016-09-09 17:15:40.381320,28.370846,-80.297147,0.040332,81.0
708,2016-09-09 17:15:45.507200,28.370852,-80.297158,0.353587,83.0
709,2016-09-09 17:15:50.620850,28.370852,-80.297158,0.529793,84.0
710,2016-09-09 17:15:55.725130,28.370852,-80.297158,0.341840,87.0
711,2016-09-09 17:16:00.834440,28.370852,-80.297158,0.349672,87.0


## Filter profiles

In [6]:
from gutils.filters import default_filter


# Assign default profiles
profiled = assign_profiles(standard)

# Default filter
filtered, count = default_filter(profiled)

print("Filtered out {} profiles".format(count))
print('Profiles : ', len(filtered.profile.unique()))
filtered.tail(20)[[
    't',
    'y',
    'x',
    'pressure',
    'profile'
]]

Filtered out 12 profiles
Profiles :  6


,t,y,x,pressure,profile
673,2016-09-09 17:13:20.424990,28.370589,-80.297060,2.140000,5.0
674,2016-09-09 17:13:22.730440,28.370594,-80.297058,1.810000,5.0
675,2016-09-09 17:13:23.743010,28.370597,-80.297057,1.735823,5.0
676,2016-09-09 17:13:26.161410,28.370604,-80.297054,1.210000,5.0
677,2016-09-09 17:13:28.297240,28.370609,-80.297052,0.910000,5.0
678,2016-09-09 17:13:28.355470,28.370609,-80.297052,0.811721,5.0
679,2016-09-09 17:13:30.441890,28.370613,-80.297050,0.650000,5.0
680,2016-09-09 17:13:33.200710,28.370621,-80.297047,0.075573,5.0
681,2016-09-09 17:13:33.826840,28.370621,-80.297047,0.200000,5.0
682,2016-09-09 17:13:37.967250,28.370630,-80.297043,0.000000,5.0


In [7]:
from gutils.filters import (
    filter_profile_depth,
    filter_profile_timeperiod,
    filter_profile_distance,
    filter_profile_number_of_points,
)

profiled = assign_profiles(standard)
original_profiles = len(profiled.profile.unique())

# Filter by Minimum Depth (meters)
below_depth = 2
filtered, rm_depth    = filter_profile_depth(profiled, below=below_depth)

# Filter by Numer of Points
num_points = 20
filtered, rm_points   = filter_profile_number_of_points(filtered, points_condition=num_points)

# Filter by Lengh of Profile (seconds)
seconds = 20
filtered, rm_time     = filter_profile_timeperiod(filtered, timespan_condition=seconds)

# Filter by Vertical Distance
distance = 18
filtered, rm_distance = filter_profile_distance(filtered, distance_condition=distance)

total_filtered = rm_depth + rm_points + rm_time + rm_distance
pprint([
    'Filtered {}/{} profiles'.format(total_filtered, original_profiles),
    'Depth ({}m): {}'.format(below_depth, rm_depth),
    'Points ({}): {}'.format(num_points, rm_points),
    'Time ({}s): {}'.format(seconds, rm_time),
    'Distance ({}m): {}'.format(distance, rm_distance),
])
print('Profiles : ', len(filtered.profile.unique()))
filtered.tail(20)[[
    't',
    'y',
    'x',
    'pressure',
    'profile'
]]

['Filtered 14/18 profiles',
 'Depth (2m): 7',
 'Points (20): 5',
 'Time (20s): 0',
 'Distance (18m): 2']
Profiles :  4


,t,y,x,pressure,profile
474,2016-09-09 17:07:06.192290000,28.369688,-80.297434,9.335977,3.0
475,2016-09-09 17:07:08.551030000,28.369693,-80.297432,8.820000,3.0
476,2016-09-09 17:07:10.805420000,28.369698,-80.297430,8.623350,3.0
477,2016-09-09 17:07:11.737340000,28.369701,-80.297429,8.260000,3.0
478,2016-09-09 17:07:15.416750000,28.369710,-80.297425,7.992956,3.0
479,2016-09-09 17:07:16.487920000,28.369713,-80.297424,7.640000,3.0
480,2016-09-09 17:07:19.743260000,28.369720,-80.297421,7.120000,3.0
481,2016-09-09 17:07:20.024020000,28.369722,-80.297420,7.260744,3.0
482,2016-09-09 17:07:24.493410000,28.369732,-80.297416,6.510000,3.0
483,2016-09-09 17:07:24.636350000,28.369732,-80.297416,6.665580,3.0
